In [1]:
import os
import configparser

config_path = os.path.join(os.getcwd(), '..', '..', 'config.ini')

config = configparser.ConfigParser()
config.read(config_path)

GUARDIAN_API_KEY = config.get('guardian_api', 'api_key')
print("Guardian API Key:", GUARDIAN_API_KEY)


NoSectionError: No section: 'guardian_api'

In [3]:
import os
import configparser

config_path = os.path.join(os.getcwd(), '..', 'config.ini')

config = configparser.ConfigParser()
config.read(config_path)

GUARDIAN_API_KEY = config.get('guardian_api', 'api_key')
print("Guardian API Key:", GUARDIAN_API_KEY)

Guardian API Key: f155b662-7be4-4e65-8a36-110369468534


In [6]:
import os
import logging
from datetime import datetime

def fetch_config():
    base_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
    config_path = os.path.join(base_dir, 'config.ini')
    config = configparser.ConfigParser()
    config.read(config_path)
    return config

def setup_logger(logger_name):
    """
    This function creates a seprate folder for each day and within each day creates all log files that are run on that day
    Args:
        logger_name(str) : The name of logger
    Returns:
        logging.Logger: logger instance that is configured
    """
    current_date = datetime.now().strftime("%Y%m%d")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_directory = os.path.join("logs", current_date)
    os.makedirs(log_directory, exist_ok=True)

    logger= logging.getLogger(logger_name)
    logger.setLevel(logging.INFO)
    log_file = os.path.join(log_directory,f"{logger_name}_{timestamp}.log")
    file_handler = logging.FileHandler(log_file)
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    file_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    return logger

In [7]:
import configparser
import requests
from datetime import datetime
import json
import os
import pandas as pd
import time

def guardian_search(search_query, from_date, to_date, max_retries=3, retry_delay=5):
        url = 'http://content.guardianapis.com/search'
        query = search_query.replace(' ', '%20')
        params = {
            'q': query,
            'from-date': from_date,
            'to-date': to_date,
            'page-size': 100,
            'api-key': 'f155b662-7be4-4e65-8a36-110369468534',
            'show-fields': 'headline,byline,sectionName,webPublicationDate'
        }
        all_results = []
        current_page = 1
        total_pages = 1
        retry_count = 0
        while current_page <= total_pages:
            params['page'] = current_page
            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                all_results.extend(data['response']['results'])
                total_pages = data['response']['pages']
                current_page += 1
            else:
                retry_count += 1
                if retry_count <= max_retries:
                    print(f"API request failed with status code {response.status_code}. Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    print(f"API request failed with status code {response.status_code} after {max_retries} retries.")
                    raise Exception(f"API request failed with status code {response.status_code} after {max_retries} retries.")

        print(f"Retrieved {len(all_results)} articles for search query: {search_query}")
        return all_results

In [8]:
search_query = "Justin Trudeau"
from_date = "2018-01-01"
to_date = datetime.now().strftime("%Y-%m-%d")
results = guardian_search(search_query, from_date, to_date)

API request failed with status code 503. Retrying in 5 seconds...
API request failed with status code 503. Retrying in 5 seconds...
API request failed with status code 503. Retrying in 5 seconds...
API request failed with status code 503 after 3 retries.


Exception: API request failed with status code 503 after 3 retries.

In [11]:
def guardian_search_version2(api_key, search_query, from_date):
    url = 'http://content.guardianapis.com/search?q='
    query = search_query.replace(' ', '%20') 
    #final_url = url + query + '&from-date=' + from_date + '&page-size=100' + '&api-key=' + api_key
    final_url = f"{url}{query}&from-date={from_date}&page-size=100&api-key={api_key}"
    response = requests.get(final_url)
    data = response.content.decode('utf-8')
    return data

guardian_json = guardian_search_version2('f155b662-7be4-4e65-8a36-110369468534', 'Justin Trudeau','2018-01-01')
guardian_json

'{"response":{"status":"ok","userTier":"developer","total":10271,"startIndex":1,"pageSize":100,"currentPage":1,"pages":103,"orderBy":"relevance","results":[{"id":"world/2023/aug/02/justin-trudeau-sophie-gregoire-trudeau-separation","type":"article","sectionId":"world","sectionName":"World news","webPublicationDate":"2023-08-02T17:16:37Z","webTitle":"Justin Trudeau and Sophie Grégoire Trudeau announce separation","webUrl":"https://www.theguardian.com/world/2023/aug/02/justin-trudeau-sophie-gregoire-trudeau-separation","apiUrl":"https://content.guardianapis.com/world/2023/aug/02/justin-trudeau-sophie-gregoire-trudeau-separation","isHosted":false,"pillarId":"pillar/news","pillarName":"News"},{"id":"world/2023/nov/09/justin-trudeau-montreal-canada-jewish-schools","type":"article","sectionId":"world","sectionName":"World news","webPublicationDate":"2023-11-09T22:56:11Z","webTitle":"Justin Trudeau condemns shots fired at two Jewish schools in Montreal","webUrl":"https://www.theguardian.com/w

In [14]:
def guardian_search(search_query, from_date, to_date, max_retries=3, retry_delay=5):
    url = 'http://content.guardianapis.com/search'
    query = search_query.replace(' ', '%20')  # Ensure spaces are replaced by '%20'
    params = {
        'q': query,
        'from-date': from_date,
        'to-date': to_date,
        'page-size': 100,
        'api-key': 'f155b662-7be4-4e65-8a36-110369468534',
        'show-fields': 'headline,byline,sectionName,webPublicationDate'
    }
    all_results = []
    current_page = 1
    total_pages = 1
    retry_count = 0
    while current_page <= total_pages:
        params['page'] = current_page
        response = requests.get(url, params=params)
        print(f"URL Requested: {response.url}")
        print(f"Status Code: {response.status_code}")

        if response.status_code == 200:
            data = response.json()
            all_results.extend(data['response']['results'])
            total_pages = data['response']['pages']
            current_page += 1
        else:
            retry_count += 1
            if retry_count <= max_retries:
                time.sleep(retry_delay)
            else:
                raise Exception(f"API request failed with status code {response.status_code} after {max_retries} retries.")

    print(f"Retrieved {len(all_results)} total articles for search query: {search_query}")
    return all_results


search_query = "Justin Trudeau"
from_date = "2018-01-01"
to_date = datetime.now().strftime("%Y-%m-%d")
results = guardian_search(search_query, from_date, to_date)

results = guardian_search(search_query, from_date, to_date)

URL Requested: http://content.guardianapis.com/search?q=Justin%2520Trudeau&from-date=2018-01-01&to-date=2024-04-24&page-size=100&api-key=f155b662-7be4-4e65-8a36-110369468534&show-fields=headline%2Cbyline%2CsectionName%2CwebPublicationDate&page=1
Status Code: 200
URL Requested: http://content.guardianapis.com/search?q=Justin%2520Trudeau&from-date=2018-01-01&to-date=2024-04-24&page-size=100&api-key=f155b662-7be4-4e65-8a36-110369468534&show-fields=headline%2Cbyline%2CsectionName%2CwebPublicationDate&page=2
Status Code: 200
URL Requested: http://content.guardianapis.com/search?q=Justin%2520Trudeau&from-date=2018-01-01&to-date=2024-04-24&page-size=100&api-key=f155b662-7be4-4e65-8a36-110369468534&show-fields=headline%2Cbyline%2CsectionName%2CwebPublicationDate&page=3
Status Code: 200
URL Requested: http://content.guardianapis.com/search?q=Justin%2520Trudeau&from-date=2018-01-01&to-date=2024-04-24&page-size=100&api-key=f155b662-7be4-4e65-8a36-110369468534&show-fields=headline%2Cbyline%2Csecti

Exception: API request failed with status code 503 after 3 retries.

In [16]:
import requests
import time
from datetime import datetime

def guardian_search(search_query, from_date, to_date, api_key, max_retries=3, retry_delay=5):
    url = 'http://content.guardianapis.com/search'
    params = {
        'q': search_query,
        'from-date': from_date,
        'to-date': to_date,
        'page-size': 100,
        'api-key': api_key,
        'show-fields': 'all',
        'show-tags': 'contributor'
    }
    all_results = []
    current_page = 1
    total_pages = 1
    retry_count = 0

    while current_page <= total_pages:
        params['page'] = current_page
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            all_results.extend(data['response']['results'])
            total_pages = data['response']['pages']
            current_page += 1
        else:
            retry_count += 1
            if retry_count <= max_retries:
                time.sleep(retry_delay)
            else:
                raise Exception(f"Failed after {max_retries} retries.")
        print(f"Processed page {current_page-1}/{total_pages}. Total results so far: {len(all_results)}.")

    print(f"Completed. Total articles retrieved: {len(all_results)}")
    return all_results

api_key = 'f155b662-7be4-4e65-8a36-110369468534'
search_query = "Justin Trudeau"
from_date = "2018-01-01"
to_date = datetime.now().strftime("%Y-%m-%d")
articles = guardian_search(search_query, from_date, to_date, api_key)


Processed page 1/103. Total results so far: 100.
Processed page 2/103. Total results so far: 200.
Processed page 3/103. Total results so far: 300.
Processed page 4/103. Total results so far: 400.
Processed page 5/103. Total results so far: 500.
Processed page 6/103. Total results so far: 600.
Processed page 7/103. Total results so far: 700.
Processed page 8/103. Total results so far: 800.
Processed page 9/103. Total results so far: 900.
Processed page 10/103. Total results so far: 1000.
Processed page 11/103. Total results so far: 1100.
Processed page 12/103. Total results so far: 1200.
Processed page 13/103. Total results so far: 1300.
Processed page 14/103. Total results so far: 1400.
Processed page 15/103. Total results so far: 1500.
Processed page 16/103. Total results so far: 1600.
Processed page 17/103. Total results so far: 1700.
Processed page 18/103. Total results so far: 1800.
Processed page 19/103. Total results so far: 1900.
Processed page 20/103. Total results so far: 2000

In [18]:
import pandas as pd

def convert_to_dataframe(api_results):
    try:
        if 'response' in api_results and 'results' in api_results['response']:
            results = api_results['response']['results']
            df = pd.DataFrame(results)
            print(f"Converted data to DataFrame with {len(df)} rows and {len(df.columns)} columns")
            return df
        else:
            print("No results found or incorrect JSON format")
            return pd.DataFrame()
    except Exception as e:
        print(f"Error processing data into DataFrame: {e}")
        return pd.DataFrame()

dataframe = convert_to_dataframe({'response': {'results': articles}})
dataframe.head()


Converted data to DataFrame with 10271 rows and 13 columns


,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,fields,tags,isHosted,pillarId,pillarName
0,world/2023/aug/02/justin-trudeau-sophie-gregoi...,article,world,World news,2023-08-02T17:16:37Z,Justin Trudeau and Sophie Grégoire Trudeau ann...,https://www.theguardian.com/world/2023/aug/02/...,https://content.guardianapis.com/world/2023/au...,{'headline': 'Justin Trudeau and Sophie Grégoi...,"[{'id': 'profile/tracey-lindeman', 'type': 'co...",False,pillar/news,News
1,world/2023/nov/09/justin-trudeau-montreal-cana...,article,world,World news,2023-11-09T22:56:11Z,Justin Trudeau condemns shots fired at two Jew...,https://www.theguardian.com/world/2023/nov/09/...,https://content.guardianapis.com/world/2023/no...,{'headline': 'Justin Trudeau condemns shots fi...,"[{'id': 'profile/leyland-cecco', 'type': 'cont...",False,pillar/news,News
2,world/2023/sep/11/canada-pm-justin-trudeau-stu...,article,world,World news,2023-09-11T00:21:12Z,Canada PM Justin Trudeau stuck in India after ...,https://www.theguardian.com/world/2023/sep/11/...,https://content.guardianapis.com/world/2023/se...,{'headline': 'Canada PM Justin Trudeau stuck i...,[],False,pillar/news,News
3,world/2024/apr/03/canada-mikmaw-fishermen-left...,article,world,World news,2024-04-03T18:41:55Z,Canada: Indigenous fishermen left to walk shoe...,https://www.theguardian.com/world/2024/apr/03/...,https://content.guardianapis.com/world/2024/ap...,{'headline': 'Canada: Indigenous fishermen lef...,"[{'id': 'profile/leyland-cecco', 'type': 'cont...",False,pillar/news,News
4,world/2024/feb/29/brian-mulroney-canadian-pm-d...,article,world,World news,2024-02-29T23:35:00Z,"Brian Mulroney, former Canadian PM, dies aged 84",https://www.theguardian.com/world/2024/feb/29/...,https://content.guardianapis.com/world/2024/fe...,"{'headline': 'Brian Mulroney, former Canadian ...","[{'id': 'profile/leyland-cecco', 'type': 'cont...",False,pillar/news,News


In [22]:
import requests
import time

def guardian_search(search_query, from_date, to_date, api_key, max_retries=3, retry_delay=5):
    url = 'http://content.guardianapis.com/search'
    session = requests.Session()  # Using session for connection pooling
    params = {
        'q': search_query,
        'from-date': from_date,
        'to-date': to_date,
        'page-size': 100,
        'api-key': api_key,
        'show-fields': 'headline,byline,sectionName,webPublicationDate'
    }
    all_results = []
    current_page = 1
    total_pages = 1
    retry_count = 0

    while current_page <= total_pages:
        params['page'] = current_page
        response = session.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            all_results.extend(data['response']['results'])
            total_pages = data['response']['pages']
            print(f"Processed page {current_page}/{total_pages}. Total results so far: {len(all_results)}.")
            current_page += 1
            retry_count = 0  # Reset retry count after a successful request
        else:
            print(f"Error {response.status_code}: {response.text}")
            retry_count += 1
            if retry_count <= max_retries:
                time.sleep(retry_delay * retry_count)  # Exponential back-off
            else:
                print("Max retries reached. Stopping.")
                break 

    return all_results


api_key = 'f155b662-7be4-4e65-8a36-110369468534'
search_query = "Justin Trudeau"
from_date = "2018-01-01"
to_date = datetime.now().strftime("%Y-%m-%d")
articles2 = guardian_search(search_query, from_date, to_date, api_key)


import pandas as pd

def convert_to_dataframe(api_results):
    try:
        if 'response' in api_results and 'results' in api_results['response']:
            results = api_results['response']['results']
            df = pd.DataFrame(results)
            print(f"Converted data to DataFrame with {len(df)} rows and {len(df.columns)} columns")
            return df
        else:
            print("No results found or incorrect JSON format")
            return pd.DataFrame()
    except Exception as e:
        print(f"Error processing data into DataFrame: {e}")
        return pd.DataFrame()

dataframe = convert_to_dataframe({'response': {'results': articles2}})
dataframe.head()

Processed page 1/103. Total results so far: 100.
Processed page 2/103. Total results so far: 200.
Processed page 3/103. Total results so far: 300.
Processed page 4/103. Total results so far: 400.
Processed page 5/103. Total results so far: 500.
Processed page 6/103. Total results so far: 600.
Processed page 7/103. Total results so far: 700.
Processed page 8/103. Total results so far: 800.
Processed page 9/103. Total results so far: 900.
Processed page 10/103. Total results so far: 1000.
Processed page 11/103. Total results so far: 1100.
Processed page 12/103. Total results so far: 1200.
Processed page 13/103. Total results so far: 1300.
Processed page 14/103. Total results so far: 1400.
Processed page 15/103. Total results so far: 1500.
Processed page 16/103. Total results so far: 1600.
Processed page 17/103. Total results so far: 1700.
Processed page 18/103. Total results so far: 1800.
Processed page 19/103. Total results so far: 1900.
Processed page 20/103. Total results so far: 2000

In [24]:
import pandas as pd

def convert_to_dataframe(api_results):
    try:
        if 'response' in api_results and 'results' in api_results['response']:
            results = api_results['response']['results']
            df = pd.DataFrame(results)
            print(f"Converted data to DataFrame with {len(df)} rows and {len(df.columns)} columns")
            return df
        else:
            print("No results found or incorrect JSON format")
            return pd.DataFrame()
    except Exception as e:
        print(f"Error processing data into DataFrame: {e}")
        return pd.DataFrame()

dataframe = convert_to_dataframe({'response': {'results': articles2}})
dataframe.head()

Converted data to DataFrame with 10272 rows and 12 columns


,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,fields,isHosted,pillarId,pillarName
0,world/2023/aug/02/justin-trudeau-sophie-gregoi...,article,world,World news,2023-08-02T17:16:37Z,Justin Trudeau and Sophie Grégoire Trudeau ann...,https://www.theguardian.com/world/2023/aug/02/...,https://content.guardianapis.com/world/2023/au...,{'headline': 'Justin Trudeau and Sophie Grégoi...,False,pillar/news,News
1,world/2023/nov/09/justin-trudeau-montreal-cana...,article,world,World news,2023-11-09T22:56:11Z,Justin Trudeau condemns shots fired at two Jew...,https://www.theguardian.com/world/2023/nov/09/...,https://content.guardianapis.com/world/2023/no...,{'headline': 'Justin Trudeau condemns shots fi...,False,pillar/news,News
2,world/2023/sep/11/canada-pm-justin-trudeau-stu...,article,world,World news,2023-09-11T00:21:12Z,Canada PM Justin Trudeau stuck in India after ...,https://www.theguardian.com/world/2023/sep/11/...,https://content.guardianapis.com/world/2023/se...,{'headline': 'Canada PM Justin Trudeau stuck i...,False,pillar/news,News
3,world/2024/apr/03/canada-mikmaw-fishermen-left...,article,world,World news,2024-04-03T18:41:55Z,Canada: Indigenous fishermen left to walk shoe...,https://www.theguardian.com/world/2024/apr/03/...,https://content.guardianapis.com/world/2024/ap...,{'headline': 'Canada: Indigenous fishermen lef...,False,pillar/news,News
4,world/2024/feb/29/brian-mulroney-canadian-pm-d...,article,world,World news,2024-02-29T23:35:00Z,"Brian Mulroney, former Canadian PM, dies aged 84",https://www.theguardian.com/world/2024/feb/29/...,https://content.guardianapis.com/world/2024/fe...,"{'headline': 'Brian Mulroney, former Canadian ...",False,pillar/news,News


In [49]:
import pandas as pd
import numpy as np

def count_trudeau_articles_claude(df):
    trudeau_articles = df[df['webTitle'].str.contains('Justin Trudeau', case=False)]
    publication_dates = pd.to_datetime(trudeau_articles['webPublicationDate']).dt.date

    start_date = pd.to_datetime('2018-01-01').date()
    end_date = pd.to_datetime('today').date()
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

    article_counts = pd.DataFrame({'Date': all_dates})
    article_counts['No. of articles'] = article_counts['Date'].map(publication_dates.value_counts()).fillna(0).astype(int)

    average_articles_per_day = article_counts['No. of articles'].mean()

    return article_counts[['Date', 'No. of articles']], average_articles_per_day


import pandas as pd

def count_articles_by_date_complete_gpt(df):
    """
    Counts the number of articles about Justin Trudeau per day, including days with zero articles,
    after filtering for articles specifically mentioning 'Justin Trudeau'.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing articles with a 'webPublicationDate' column and other identifying information.
    
    Returns:
    pandas.DataFrame: DataFrame with columns 'Date' and 'No. of articles', listing article counts for each day, including zeros.
    """

    filtered_df = df[df['webTitle'].str.contains("Justin Trudeau", case=False, na=False)]

    filtered_df['Date'] = pd.to_datetime(filtered_df['webPublicationDate']).dt.date
    
    article_counts = filtered_df.groupby('Date').size().reset_index(name='No. of articles')
    article_counts['Date'] = pd.to_datetime(article_counts['Date']) 
    
    start_date = pd.to_datetime('2018-01-01').date()
    end_date = pd.to_datetime('today').date()
    all_dates = pd.DataFrame({'Date': pd.date_range(start=start_date, end=end_date, freq='D')})
    all_dates['Date'] = pd.to_datetime(all_dates['Date'])  
    
    complete_article_counts = pd.merge(all_dates, article_counts, on='Date', how='left')
    complete_article_counts['No. of articles'].fillna(0, inplace=True)
    complete_article_counts['No. of articles'] = complete_article_counts['No. of articles'].astype(int)
    
    return complete_article_counts



In [52]:
# count_gpt = count_articles_by_date_complete_gpt(dataframe)
# count_gpt.head()
count_claude, average_claude = count_trudeau_articles_claude(dataframe)
count_claude.head()
#print(f"\nAverage number of articles per day: {average_claude:.2f}")


,Date,No. of articles
0,2018-01-01,0
1,2018-01-02,0
2,2018-01-03,0
3,2018-01-04,0
4,2018-01-05,0


In [48]:
def calculate_average_articles_gpt(df):
    """
    Calculates the average number of articles per day over the entire date range in the DataFrame.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns 'Date' and 'No. of articles'.
    
    Returns:
    float: The average number of articles per day.
    """
    average_articles = df['No. of articles'].mean()
    return average_articles


average_articles_per_day = calculate_average_articles_gpt(count_gpt)
print(f"The average number of articles per day from 2018-01-01 to today is: {average_articles_per_day:.2f}")


The average number of articles per day from 2018-01-01 to today is: 0.03
